In [12]:
import tkinter as tk
from tkinter import messagebox

# Function that checks if all checkboxes are selected and closes the window.
def Check_If_All_Selected(Window: tk.Tk, Checkbox_Variables: list[tk.IntVar]) -> None:
    if all(Checkbox_Variable.get() == 1 for Checkbox_Variable in Checkbox_Variables):  # If all checkboxes are selected.
        Window.destroy()  # Close the window.
        Main_Window.destroy()

# Function to add a new input field dynamically with a remove button.
def Add_New_Input_With_Remove(Main_Window: tk.Tk, Input_Entries: list[tk.Frame]) -> None:
    New_Input_Frame = tk.Frame(Main_Window)  # Create a frame for the new input and button.
    
    New_Input_Label = tk.Label(New_Input_Frame, text = f"Task:", font=("Calibri Light", 13))
    New_Input_Label.pack(side='left', padx=5, pady=5)  # Place label on the left side.

    New_Input_Text = tk.Text(New_Input_Frame, font=("Calibri Light", 12), height=1,  width=50)  # Set height and width
    New_Input_Text.pack(side='left', padx = 10, pady = 5)
    
    Remove_Button = tk.Button(New_Input_Frame, text = "x", font=("Calibri Light", 12), width=3,
                              command = lambda: Remove_Input(Main_Window, New_Input_Frame, Input_Entries))
    Remove_Button.pack(side='left', padx = 5, pady = 5)  # Place remove button next to the entry.
    
    New_Input_Frame.pack()  # Pack the frame containing the input and button.
    Input_Entries.append(New_Input_Frame)  # Store the new frame with entry.

# Function to remove an input field dynamically.
def Remove_Input(Main_Window: tk.Tk, Input_Frame: tk.Frame, Input_Entries: list[tk.Frame]) -> None:
    Input_Frame.destroy()  # Destroy the input frame.
    Input_Entries.remove(Input_Frame)  # Remove the frame from the list.

# Function that handles input collection and opens the checklist window.
def Collect_Inputs(Input_Entries: list[tk.Frame]) -> None:
    Inputs = [Input_Frame.winfo_children()[1].get("1.0", "end-1c") for Input_Frame in Input_Entries]  # Collect all input values.
    if all(Inputs):  # Proceed if all inputs are filled.
        Create_Checklist_Window(Inputs)
    else:
        messagebox.showwarning("Input Error", "Please fill in all fields.")

# Function that creates a new window with checkboxes from input values.
def Create_Checklist_Window(Inputs: list[str]) -> None:
    Checklist_Window = tk.Toplevel()  # Create new window.
    Checklist_Window.title("Checklist Window")
    Checklist_Window.geometry("1200x500")
    Checklist_Window.geometry("+{}+{}".format(int(Checklist_Window.winfo_screenwidth() / 2 - 600), int(Checklist_Window.winfo_screenheight() / 2 - 250)))  # Center the window.

    Checklist_Window.attributes('-topmost', True)  # Keep the window on top.
    Checklist_Window.attributes('-toolwindow', True)  # Disable minimize.

    Checklist_Window.focus_force()  # Force focus on the window.
    Checklist_Window.grab_set()  # Prevent clicking outside the window.

    Checkbox_Variables = []
    
    for Input_Value in Inputs:  # Create a checkbox for each input.
        Checkbox_Variable = tk.IntVar()
        Checkbox_Variables.append(Checkbox_Variable)
        Checkbox = tk.Checkbutton(Checklist_Window, text = Input_Value, variable = Checkbox_Variable)
        Checkbox.pack()
    
    # Button that checks if all checkboxes are selected.
    Accept_Button = tk.Button(Checklist_Window, text = "Check All", command = lambda: Check_If_All_Selected(Checklist_Window, Checkbox_Variables))
    Accept_Button.pack()


# Main window setup.
Main_Window = tk.Tk()
Main_Window.title("Input Window")
Main_Window.geometry("1200x500")
Main_Window.geometry("+{}+{}".format(int(Main_Window.winfo_screenwidth() / 2 - 600), int(Main_Window.winfo_screenheight() / 2 - 250)))  # Center the window.

Main_Window.attributes('-topmost', True)  # Keep the window on top.
Main_Window.attributes('-toolwindow', True)  # Disable minimize.

Main_Window.focus_force()  # Force focus on the window.
Main_Window.grab_set()  # Prevent clicking outside the window.

# Frame to hold all dynamically added input fields.
Input_Frame = tk.Frame(Main_Window)
Input_Frame.pack(fill = 'x', padx = 10, pady = 15)

Input_Entries = []  # List to store dynamically added input fields.

# Initial input fields.
Add_New_Input_With_Remove(Input_Frame, Input_Entries)

# Button to add new input fields dynamically.
Add_Input_Button = tk.Button(Main_Window, text = "+", command = lambda: Add_New_Input_With_Remove(Input_Frame, Input_Entries))
Add_Input_Button.pack(pady = 5)

# Button to accept inputs and create the checklist window.
Accept_Button = tk.Button(Main_Window, text = "Accept", command = lambda: Collect_Inputs(Input_Entries))
Accept_Button.pack()

Main_Window.mainloop()
